In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train.csv to train.csv


In [ ]:
df = pd.read_csv("train.csv")
df.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [ ]:
# Check total and percentage of missing values
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

# Combine and show them together
missing_summary = pd.DataFrame({
    'Missing_Values': missing_values,
    'Percent': missing_percent
})

# Display
missing_summary[missing_summary['Missing_Values'] > 0]


,Missing_Values,Percent
Mileage,2,0.034206
Engine,36,0.615700
Power,36,0.615700
Seats,38,0.649906
New_Price,5032,86.061228


In [ ]:
# Handle missing values
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')
df['Engine'] = df['Engine'].fillna(df['Engine'].median())
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')
df['Power'] = df['Power'].fillna(df['Power'].median())
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].median())


# Check for missing values in all columns
df.isnull().sum()



,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Owner_Type,0
Mileage,0
Engine,0
Power,0
Seats,0


### Part (a) – Handling Missing Values

We began by analyzing all columns for missing data. The following strategy was used:

- **New_Price** had over 85% missing values, so we dropped the column to avoid biased imputations and poor data quality.
- **Seats** was imputed using the **mode**, as it’s a low-range categorical numeric feature (e.g., 5, 7).
- **Engine**, **Power**, and **Mileage** had textual units, so we first converted them to numeric values using `pd.to_numeric()` and then imputed missing values using the **median**. This approach is robust to outliers, which are common in vehicle specs.
- After this step, we verified that **all columns have 0 missing values**, ensuring the dataset is now complete and ready for further processing.

This approach balances data integrity with minimal loss of information.


In [ ]:
# Remove units from columns

# Remove ' CC' from Engine
df['Engine'] = df['Engine'].astype(str).str.replace(' CC', '', regex=False)

# Remove ' bhp' from Power
df['Power'] = df['Power'].astype(str).str.replace(' bhp', '', regex=False)

# Remove ' kmpl' and ' km/kg' from Mileage
df['Mileage'] = df['Mileage'].astype(str).str.replace(' kmpl', '', regex=False)
df['Mileage'] = df['Mileage'].str.replace(' km/kg', '', regex=False)

# Output: show cleaned values
df[['Engine', 'Power', 'Mileage']].head()


,Engine,Power,Mileage
0,1582.0,126.2,19.67
1,1199.0,88.7,13.0
2,1248.0,88.76,20.77
3,1968.0,140.8,15.2
4,1461.0,63.1,23.08


### Part (b) – Removing Units from Attributes

Several columns had numeric values stored as text with units:
- `Engine` had values like "1197 CC"
- `Power` had values like "82 bhp"
- `Mileage` had values like "18.5 kmpl" or "20.8 km/kg"

We used `.str.replace()` to remove the units and retain only numeric values. This allows for proper numerical operations like imputation, scaling, and modeling.

This transformation was essential because machine learning models require numeric inputs, and unit strings can interfere with calculations. We skipped `New_Price` because it was dropped earlier due to excessive missing values.


In [ ]:
# Re-load original data to fetch Fuel_Type and Transmission
original_df = pd.read_csv("train.csv")

# Extract only the columns we need
df['Fuel_Type'] = original_df['Fuel_Type']
df['Transmission'] = original_df['Transmission']

# Apply one-hot encoding again
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# Output check
df.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.2,5.0,12.50,False,False,True,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.0,1199.0,88.7,5.0,4.50,False,True,True,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2,1968.0,140.8,5.0,17.74,False,False,False,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.1,5.0,3.50,False,False,True,False,False,True


### Part (c) – One-Hot Encoding of Categorical Variables

The categorical columns `Fuel_Type` and `Transmission` were reintroduced from the original dataset to simulate the one-hot encoding step.

They were converted into numerical columns using `pd.get_dummies()` with `drop_first=True` to:
- Avoid the dummy variable trap
- Ensure models don’t misinterpret categorical variables as ordinal
- Make them usable in numerical modeling

This step ensures each category is represented as an independent binary column.


In [ ]:
# Create new feature: Car_Age
current_year = pd.Timestamp.now().year
df['Car_Age'] = current_year - df['Year']

# Output check – show all columns including Car_Age
df.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.2,5.0,12.50,False,False,True,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.0,1199.0,88.7,5.0,4.50,False,True,True,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2,1968.0,140.8,5.0,17.74,False,False,False,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.1,5.0,3.50,False,False,True,False,False,True,12


### Part (d) – Creating a New Feature: Car Age

We created a new column `Car_Age`in the last  by subtracting the car's manufacturing year (`Year`) from the current year. This tells us how old each car is.

Including this in the full dataset allows better analysis and interpretation when combined with all other car attributes.

Why it's useful:
- Car age is a key factor in pricing and performance.
- Makes it easier to group, filter, or model based on vehicle age.
- Equivalent to `mutate()` in R for creating new columns based on existing ones.


In [ ]:
# Select only specific columns of interest
selected_df = df[['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type_Petrol', 'Transmission_Manual', 'Car_Age', 'Price']]

# Output check
selected_df.head()


,Name,Location,Year,Kilometers_Driven,Fuel_Type_Petrol,Fuel_Type_Petrol,Transmission_Manual,Transmission_Manual,Car_Age,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,False,False,True,True,10,12.50
1,Honda Jazz V,Chennai,2011,46000,True,True,True,True,14,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,False,False,True,True,13,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,False,False,False,False,12,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,False,False,True,True,12,3.50


In [ ]:
# Select only specific columns of interest
selected_df = df[['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type_Petrol', 'Transmission_Manual', 'Car_Age', 'Price']]

# Output check
selected_df.head()


,Name,Location,Year,Kilometers_Driven,Fuel_Type_Petrol,Fuel_Type_Petrol,Transmission_Manual,Transmission_Manual,Car_Age,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,False,False,True,True,10,12.50
1,Honda Jazz V,Chennai,2011,46000,True,True,True,True,14,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,False,False,True,True,13,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,False,False,False,False,12,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,False,False,True,True,12,3.50


In [ ]:
# Ensure 'Mileage' is fully numeric before filtering
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')

# Filter: select cars with Mileage > 20
filtered_df = df[df['Mileage'] > 20]

# Output check
filtered_df[['Name', 'Mileage', 'Price']].head()


,Name,Mileage,Price
2,Maruti Ertiga VDI,20.77,6.00
4,Nissan Micra Diesel XV,23.08,3.50
6,Volkswagen Vento Diesel Comfortline,20.54,5.20
7,Tata Indica Vista Quadrajet LS,22.30,1.95
8,Maruti Ciaz Zeta,21.56,9.95


In [ ]:
# Rename column
df_renamed = df.rename(columns={'Price': 'Selling_Price'})

# Output check
df_renamed[['Name', 'Selling_Price']].head()


,Name,Selling_Price
0,Hyundai Creta 1.6 CRDi SX Option,12.50
1,Honda Jazz V,4.50
2,Maruti Ertiga VDI,6.00
3,Audi A4 New 2.0 TDI Multitronic,17.74
4,Nissan Micra Diesel XV,3.50


In [ ]:
# Create new column: Price per year of car age
df_renamed['Price_Per_Year'] = df_renamed['Selling_Price'] / df_renamed['Car_Age']

# Output check
df_renamed[['Name', 'Car_Age', 'Selling_Price', 'Price_Per_Year']].head()


,Name,Car_Age,Selling_Price,Price_Per_Year
0,Hyundai Creta 1.6 CRDi SX Option,10,12.50,1.250000
1,Honda Jazz V,14,4.50,0.321429
2,Maruti Ertiga VDI,13,6.00,0.461538
3,Audi A4 New 2.0 TDI Multitronic,12,17.74,1.478333
4,Nissan Micra Diesel XV,12,3.50,0.291667


In [ ]:
# Sort by Selling Price (Descending)
sorted_df = df_renamed.sort_values(by='Selling_Price', ascending=False)

# Output check
sorted_df[['Name', 'Selling_Price']].head()


,Name,Selling_Price
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,160.00
5620,Lamborghini Gallardo Coupe,120.00
5752,Jaguar F Type 5.0 V8 S,100.00
1457,Land Rover Range Rover Sport SE,97.07
1917,BMW 7 Series 740Li,93.67


In [ ]:
# Group by Location and summarize price
summary_df = df_renamed.groupby('Location')['Selling_Price'].agg(['mean', 'median', 'count']).reset_index()

# Rename columns
summary_df.columns = ['Location', 'Avg_Price', 'Median_Price', 'Total_Cars']

# Output check
summary_df.head()


,Location,Avg_Price,Median_Price,Total_Cars
0,Ahmedabad,8.567248,6.000,218
1,Bangalore,13.482670,6.970,352
2,Chennai,7.958340,4.725,476
3,Coimbatore,15.160206,8.360,631
4,Delhi,9.881944,5.700,540


### Part (e) – Select, Filter, Rename, Mutate, Arrange, and Group By (Python Equivalents)

We performed equivalent operations in Python using pandas:

- **Select**: Retrieved key columns like Name, Location, Year, etc.
- **Filter**: Kept only rows with mileage greater than 20 kmpl.
- **Rename**: Renamed the `Price` column to `Selling_Price`.
- **Mutate**: Created a new column `Price_Per_Year` (Selling_Price ÷ Car_Age).
- **Arrange**: Sorted the data in descending order of Selling Price.
- **Group By & Summarize**: Grouped cars by Location and calculated average, median, and count of prices.

These operations demonstrate structured data wrangling and transformation similar to `dplyr` or `tidyverse` in R.


In [ ]:
from google.colab import files

# 1. SELECTED columns
selected_df = df[['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type_Petrol', 'Transmission_Manual', 'Car_Age', 'Price']]
selected_df.to_csv('selected_result.csv', index=False)

# 2. FILTERED (Mileage > 20)
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
filtered_df = df[df['Mileage'] > 20]
filtered_df.to_csv('filtered_result.csv', index=False)

# 3. RENAMED (Price → Selling_Price)
df_renamed = df.rename(columns={'Price': 'Selling_Price'})
df_renamed.to_csv('renamed_result.csv', index=False)

# 4. MUTATED (Price per Year)
df_renamed['Price_Per_Year'] = df_renamed['Selling_Price'] / df_renamed['Car_Age']
df_renamed.to_csv('mutated_result.csv', index=False)

# 5. ARRANGED (Sorted by Selling_Price)
sorted_df = df_renamed.sort_values(by='Selling_Price', ascending=False)
sorted_df.to_csv('arranged_result.csv', index=False)

# 6. FINAL full dataset
df.to_csv('processed_used_cars.csv', index=False)

# DOWNLOAD all CSVs
files.download('selected_result.csv')
files.download('filtered_result.csv')
files.download('renamed_result.csv')
files.download('mutated_result.csv')
files.download('arranged_result.csv')
files.download('processed_used_cars.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Exporting Result Files for Part (e) – Submission Justification

As part of the assignment's final requirement, the results of all data transformation steps in Part (e) were saved into separate `.csv` files and downloaded for submission. This was done to clearly demonstrate and preserve the output of each individual operation.

Here is the breakdown of each file and its purpose:

1. **selected_result.csv** – Contains only selected columns such as Name, Location, Year, Kilometers_Driven, Fuel_Type, Transmission, Car_Age, and Price.  
   ➤ This demonstrates the `select` operation.

2. **filtered_result.csv** – Contains cars with `Mileage` greater than 20 kmpl.  
   ➤ This shows the `filter` operation.

3. **renamed_result.csv** – The original `Price` column was renamed to `Selling_Price`.  
   ➤ This represents the `rename` operation.

4. **mutated_result.csv** – A new derived column `Price_Per_Year` was added by dividing `Selling_Price` by `Car_Age`.  
   ➤ This reflects the `mutate` operation.

5. **arranged_result.csv** – The dataset was sorted in descending order of `Selling_Price`.  
   ➤ This shows the `arrange` operation.

6. **processed_used_cars.csv** – The final, cleaned, and transformed dataset after all preprocessing and enhancements.  
   ➤ This file represents the complete processed data ready for analysis or modeling.


